In [13]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.utils import plot_model
import visualkeras



In [15]:
# 1. Load and preprocess MNIST
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test  = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), 28*28))
x_test  = x_test.reshape((len(x_test),  28*28))

In [19]:
# 2. Build the autoencoder model
input_img = Input(shape=(784,))
x = Dense(1024, activation='relu')(input_img)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(8, activation='relu')(x)
encoded   = Dense(2, activation='sigmoid')(x)   # Encoded representation (dimension reduction)
y = Dense(8, activation='relu')(encoded)
y = Dense(64, activation='relu')(y)
y = Dense(256, activation='relu')(y)
y = Dense(1024, activation='relu')(y)
decoded   = Dense(784, activation='sigmoid')(y) # Output layer

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.save('mnist_feedforward.keras')
summary(autoencoder)

NameError: name 'summary' is not defined

In [ ]:
# 3. Train the autoencoder (just a few epochs for demo)
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# 4. Visualize the model architecture
img = visualkeras.layered_view(
    autoencoder,  
    to_file=None, 
    scale_xy=2, 
    scale_z=1, 
    max_z=6,  # thin Z-stacking for shallow net
    draw_volume=False,  # option: True for more 3D look
)
plt.figure(figsize=(8,3))
plt.imshow(img)
plt.axis('off')
plt.show()

In [31]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime

# Set a folder to store logs (each run should have a unique folder)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


writer = tf.summary.create_file_writer('logs/graph_architecture')
tf.summary.trace_on(graph=True, profiler=True)
# Call your model once to "build" it
autoencoder(np.zeros((1,784)))
with writer.as_default():
    tf.summary.trace_export(
        name="autoencoder_trace",
        step=0,
        profiler_outdir='logs/graph_architecture'
    )

2025-11-05 16:58:45.755569: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-11-05 16:58:45.755587: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2025-11-05 16:58:45.767246: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-11-05 16:58:45.767569: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


In [65]:
from graphviz import Digraph

# Layer definitions: (name, shape, units, activation)
layer_data = [
    ('Input', 'oval', 784, ''),
    ('Dense1', 'box', 1024, 'relu'),
    ('Dense2', 'box', 256, 'relu'),
    ('Dense3', 'box', 64, 'relu'),
    ('Dense4', 'box', 8, 'relu'),
    ('Bottleneck', 'ellipse', 2, 'sigmoid'),
    ('Dense5', 'box', 8, 'relu'),
    ('Dense6', 'box', 64, 'relu'),
    ('Dense7', 'box', 256, 'relu'),
    ('Dense8', 'box', 1024, 'relu'),
    ('Output', 'oval', 784, 'sigmoid'),
]

# Explicit color assignments
input_output_color = "#4285F4"   # Blue
bottleneck_color   = "#9874C5"   # Purple

dense_colors = {
    'Dense1': "#34A853",    # Green
    'Dense2': "#FBBC05",    # Yellow
    'Dense3': "#FFA500",    # Orange
    'Dense4': "#EA4335",    # Red
    'Dense5': "#EA4335",    # Red          # symmetric with Dense4
    'Dense6': "#FFA500",    # Orange       # symmetric with Dense3
    'Dense7': "#FBBC05",    # Yellow       # symmetric with Dense2
    'Dense8': "#34A853",    # Green        # symmetric with Dense1
}

# Scaling height based on number of units
max_units = max(layer[2] for layer in layer_data)
min_units = min(layer[2] for layer in layer_data)

def scale_height(units, scale_min=0.7, scale_max=2.5):
    if max_units == min_units:
        return (scale_min + scale_max) / 2
    return scale_min + (scale_max - scale_min) * (units - min_units) / (max_units - min_units)

fixed_width = '0.2'  # inches

dot = Digraph('Autoencoder', format='png')
dot.attr(rankdir='LR', dpi='300', size='14,5')

# Draw nodes
for idx, (name, shape, units, activation) in enumerate(layer_data):
    label = f'{name}\n{units}'
    if activation:
        label += f'\n{activation}'
    height = str(round(scale_height(units), 2))
    if name == 'Input' or name == 'Output':
        fillcolor = input_output_color
    elif name == 'Bottleneck':
        fillcolor = bottleneck_color
    else:
        fillcolor = dense_colors[name]
    dot.node(
        name,
        label=label,
        shape=shape,
        width=fixed_width,
        height=height,
        style='filled',
        fillcolor=fillcolor
    )

# Draw edges
for i in range(len(layer_data) - 1):
    dot.edge(layer_data[i][0], layer_data[i+1][0])

dot.render('manual_autoencoder_custom_colors', view=True)

'manual_autoencoder_custom_colors.png'

In [3]:
from graphviz import Digraph

dot = Digraph('Autoencoder', format='png')
dot.attr(rankdir='TB', splines='ortho', bgcolor='white')
dot.attr('node', fontname='Helvetica', fontsize='18', style='filled', color='gray70')

dot.node('input',      'Input',  shape='rect', width='2.2', height='0.7', fillcolor='#99ccff', penwidth='2', fixedsize='true')
dot.node('output',     'Output', shape='rect', width='2.2', height='0.7', fillcolor='#99ccff', penwidth='2', fixedsize='true')

# Encoder trapezoid (polygon, vertical with orientation=0)
dot.node('encoder', 'Encoder',
         shape='polygon', sides='4',
         orientation='0',
         distortion='0.35',
         width='1.3', height='1.0',
         fillcolor='#a0e6a0', penwidth='2', fixedsize='true')

# Decoder mirrored trapezoid (polygon, vertical with orientation=90)
dot.node('decoder', 'Decoder',
         shape='polygon', sides='4',
         orientation='180',
         distortion='0.35',
         width='1.3', height='1.0',
         fillcolor='#ffe698', penwidth='2', fixedsize='true')

dot.node('bottleneck', 'Bottleneck', shape='square', width='0.55', height='0.55',
         fillcolor='#d4bbff', penwidth='2', color='#9874C5', fixedsize='true')

dot.edge('input',      'encoder',     penwidth='2', arrowsize='1.3')
dot.edge('encoder',    'bottleneck',  penwidth='2', arrowsize='1.3')
dot.edge('bottleneck', 'decoder',     penwidth='2', arrowsize='1.3')
dot.edge('decoder',    'output',      penwidth='2', arrowsize='1.3')

dot.render('autoencoder_vertical', view=True)

'autoencoder_vertical.png'